レース環境情報をスクレイピングで取得

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from tqdm import tqdm
import datetime

pd.options.display.width = 500
pd.options.display.column_space = 200

In [2]:
# race_id = "202104020101"
# url = "https://race.netkeiba.com/race/result.html?race_id=" + race_id
# html = requests.get(url)
# html.encoding= "EUC_JP"
# soup = BeautifulSoup(html.text, "html.parser")

In [3]:
# 日付 = soup.find("dd",class_='Active')
# 日付.find("a").get_text()

In [4]:
# レース情報 = soup.find("div", class_="RaceList_Item02") 

In [5]:
# レース名 = レース情報.find("div",class_="RaceName").get_text()
# re.findall(r"\w+",レース名)

In [6]:
# レース環境 = レース情報.find("div",class_="RaceData01").get_text()
# re.findall(r"\w+",レース環境)

----

スクレイピングしながらデータの成形

In [7]:
df = pd.read_pickle('df.pickle')
race_id_list = df.index.unique()
race_id_list

Index(['202101010101', '202101010102', '202102010101', '202102010102', '202103010101', '202103010102', '202104010101', '202104010102', '202105010101', '202105010102',
       ...
       '202109040102', '202109040201', '202109040202', '202109040301', '202110040101', '202110040102', '202110040201', '202110040202', '202110040301', '202110040302'], dtype='object', length=173)

df.pickleに保存したdataframeのindexを参照してして全てのレース環境情報をスクレイピング

In [8]:
race_infos = {}
for race_id in tqdm(race_id_list):
  url = "https://race.netkeiba.com/race/result.html?race_id=" + race_id
  html = requests.get(url)
  html.encoding= "EUC_JP"
  soup = BeautifulSoup(html.text, "html.parser")

  日付 = soup.find("dd",class_='Active')
  日付 = 日付.find("a").get_text()
  レース情報 = soup.find("div", class_="RaceList_Item02") 
  # 距離 = レース情報.find("div",class_="RaceData01").find("span").get_text()
  # 馬場 = レース情報.find("div",class_="RaceData01").find("span",class_="Item03").get_text()
  レース環境 = レース情報.find("div",class_="RaceData01").get_text()
  info = re.findall(r"\w+",レース環境)
  info[0] = info[2][:1]
  info[1] = info[2][1:]
  info.pop(2)
  info_dict = {}
  # print(info)
  info_dict["date"] = f"{race_id[:4]}年{日付[:-3]}"

  for text in info:
    # print(text)
    if text in ["芝","ダ"]:
      # print("芝付き")
      info_dict["race_type"] = text
    if "障" in text:
      info_dict["race_type"] = "障害"
    if "m" in text:
      # print("mつき")
      info_dict["course_ren"] = re.findall(r"\d+",text)[0]
    if text in ["雨","小雨","小雪","雪","晴","曇"]:
      # print("天気")
      info_dict["weather"] = text
    # if "年" in text:
    #   info_dict["data"] = text
    if text in ["良","不良","重","稍","不"]:
      # print("馬場")
      info_dict["ground_state"] = text


  race_infos[race_id] = info_dict
  time.sleep(.5)

100%|██████████| 173/173 [02:30<00:00,  1.15it/s]


In [9]:
race_infos_df = pd.DataFrame(race_infos).T


In [10]:
results = pd.read_pickle("df.pickle")

In [11]:
results_addinfo = results.merge(race_infos_df,left_index=True,right_index=True,how="inner")

In [12]:
results_addinfo.columns

Index(['着順', '枠', '馬番', '馬名', '斤量', '騎手', '人気', '性', '齢', '体重', '体重変化', 'date', 'race_type', 'course_ren', 'weather', 'ground_state'], dtype='object')

In [13]:
pd.set_option('display.max_rows',None)
# results_addinfo["ground_state"].value_counts()
results_addinfo.isnull().sum()

着順              0
枠               0
馬番              0
馬名              0
斤量              0
騎手              0
人気              0
性               0
齢               0
体重              0
体重変化            0
date            0
race_type       0
course_ren      0
weather         0
ground_state    0
dtype: int64

日付データの成形

In [14]:
results_addinfo["date"] = pd.to_datetime(results_addinfo["date"],format="%Y年%m月%d日")
results_addinfo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2416 entries, 202101010101 to 202110040302
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   着順            2416 non-null   int64         
 1   枠             2416 non-null   float64       
 2   馬番            2416 non-null   object        
 3   馬名            2416 non-null   object        
 4   斤量            2416 non-null   float64       
 5   騎手            2416 non-null   object        
 6   人気            2416 non-null   float64       
 7   性             2416 non-null   object        
 8   齢             2416 non-null   int64         
 9   体重            2416 non-null   int64         
 10  体重変化          2416 non-null   int64         
 11  date          2416 non-null   datetime64[ns]
 12  race_type     2416 non-null   object        
 13  course_ren    2416 non-null   object        
 14  weather       2416 non-null   object        
 15  ground_state  2416 non-n

In [ ]:
results_addinfo.to_pickle("results_p")

In [15]:
sorted_id_list = results_addinfo.sort_values("date").index.unique()

In [16]:
round(len(sorted_id_list) * 0.7)

121

In [17]:
# train_id_list = sorted_id_list[:round(len(sorted_id_list) * 0.7)]
# test_id_list = sorted_id_list[round(len(sorted_id_list) * 0.7):]

In [18]:
# train = results_addinfo.loc[train_id_list]
# test = results_addinfo.loc[test_id_list]

In [19]:
# len(train)

In [20]:
# len(test)

In [21]:
# len(results_addinfo)

In [22]:
def split_data(df, test_size):
  # ソート
  sorted_id_list = df.sort_values('date').index.unique()
  # データの分割
  train_id_list = sorted_id_list[:round(len(sorted_id_list) * (1-test_size))]
  test_id_list = sorted_id_list[round(len(sorted_id_list) * (1-test_size)):]
  
  train = df.loc[train_id_list]
  test = df.loc[test_id_list]
  return train, test

In [23]:
# ダミーデータの作成
results_d = pd.get_dummies(results_addinfo)
results_d["rank"] = results_d["着順"].map(lambda x: x if x < 4 else 4)
train,test = split_data(results_d,0.3)

In [24]:
train.columns

Index(['着順', '枠', '斤量', '人気', '齢', '体重', '体重変化', 'date', '馬番_1', '馬番_2',
       ...
       'course_ren_3390', 'weather_小雨', 'weather_晴', 'weather_曇', 'weather_雨', 'ground_state_不', 'ground_state_稍', 'ground_state_良', 'ground_state_重', 'rank'], dtype='object', length=2099)

In [25]:
from imblearn.under_sampling import RandomUnderSampler

rank_1 = train["rank"].value_counts()[1]
rank_2 = train["rank"].value_counts()[2]
rank_3 = train["rank"].value_counts()[3]

rus = RandomUnderSampler(sampling_strategy="auto",random_state=None,replacement=False)

X_train = train.drop(["着順","date","rank"],axis=1)
y_train = train["rank"]
X_test = test.drop(["着順","date","rank"],axis=1)
y_test = test["rank"]


X_train_rus,y_train_rus = rus.fit_resample(X_train,y_train)

In [ ]:
X_train_rus

In [27]:
y_train_rus

0      1
1      1
2      1
3      1
4      1
5      1
6      1
7      1
8      1
9      1
10     1
11     1
12     1
13     1
14     1
15     1
16     1
17     1
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     1
30     1
31     1
32     1
33     1
34     1
35     1
36     1
37     1
38     1
39     1
40     1
41     1
42     1
43     1
44     1
45     1
46     1
47     1
48     1
49     1
50     1
51     1
52     1
53     1
54     1
55     1
56     1
57     1
58     1
59     1
60     1
61     1
62     1
63     1
64     1
65     1
66     1
67     1
68     1
69     1
70     1
71     1
72     1
73     1
74     1
75     1
76     1
77     1
78     1
79     1
80     1
81     1
82     1
83     1
84     1
85     1
86     1
87     1
88     1
89     1
90     1
91     1
92     1
93     1
94     1
95     1
96     1
97     1
98     1
99     1
100    1
101    1
102    1
103    1
104    1
105    1
106    1
107    1
108    1
109    1
110    1
1

In [28]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0)
clf.fit(X_train_rus,y_train_rus)

print(clf.score(X_train,y_train),clf.score(X_test,y_test))

0.7296528173022196 0.47344461305007585
